MNIST project
=================
## __0. Data preprecessing__
* train_test_split를 이용한 train, test data 구성
* Test data와 Train data 비율 3:7


In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
MNIST= fetch_openml('mnist_784', version=1, as_frame=False)
x=MNIST.data
print(x.shape)
y=MNIST.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle='False')

(70000, 784)


## __1. supervised learning__
>   보편적 Multiclassification 분류기 적용, 앙상블을 통한 개별모델의 통합 및 효율화, 하이퍼파라메터 튜닝에 대한 기본 개념 및 API 학습
>   ## 1.1 Multiclassification
>   * 보편적인 3종의 다중분류기 (Regression, SVM, Dicision tree) 를 이용 및 정확도 분석 수행
>   
>   ### 1.1.1 Regression
>   * 개념 :

>   #### __1) Linear Regression__

In [25]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
LR=LinearRegression()
LR.fit(x_train,y_train)
y_pre=LR.predict(x_train)
y_pre_test=LR.predict(x_test)

score_train=metrics.r2_score(y_pre,y_train)
score_test=metrics.r2_score(y_pre_test,y_test)

print('The r2 score(train) is ',score_train)
print('The r2 score(test) is ',score_test)

r2score=(["Linear regression",score_test])
print(r2score)

The r2 score(train) is  0.3823097688273094
The r2 score(test) is  -4.726941154897801e-05
['Linear regression', -4.726941154897801e-05]


>   #### __2) Lasso Regression__

In [41]:
import numpy as np
from sklearn.linear_model import Lasso

Lass=Lasso(alpha=0.9)
Lass.fit(x_train,y_train)
y_pre_train=Lass.predict(x_train)
score_train=metrics.r2_score(y_pre_train,y_train)
y_pre_test=Lass.predict(x_test)
score_test=metrics.r2_score(y_pre_test,y_test)

print('The r2 score(train) is ',score_train)
print('The r2 score(test) is ',score_test)
r2score=np.append(r2score,np.array(["Lasso",score_test]))
print(r2score)

The r2 score(train) is  0.3087809136367944
The r2 score(test) is  0.3050446404257243
['Linear regression' '-4.726941154897801e-05' 'Lasso'
 '0.34459369551893004' 'Lasso' '0.30890081528355906' 'Lasso'
 '0.30890081528355906' 'Lasso' '0.3050446404257243']


>   #### __3) Logistic Regression__

In [39]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(solver='lbfgs',random_state=42)
log.fit(x_train,y_train)
y_pre_train=log.predict(x_train)
y_pre_test=log.predict(x_test)

score_train=metrics.accuracy_score(y_pre_train,y_train)
score_test=metrics.accuracy_score(y_test,y_pre_test)

print('Accuracy(train) is ',score_train)
print('Accuracy(test) is ',score_test)


Accuracy(train) is  0.9342857142857143
Accuracy(test) is  0.9161428571428571


c:\Users\poker\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# parameter study
print('Iteration for training is',log.max_iter)
print('Comparsion study')

for iter in [100, 1000, 2000]:
    log=LogisticRegression(solver='lbfgs',max_iter=iter,random_state=42)
    log.fit(x_train,y_train)
    pred_test=log.predict(x_test)
    print('Accuracy in iterate',iter,'is',metrics.accuracy_score(y_test,pred_test))


* Interation 이 증가하더라도 뚜렷한 정확도 상승 기대는 어려움
    - iter 100 : 91.63%
    - iter 1000 : 91.43%
    - iter 2000 : 91.212%
    
    
* Itertation 이 1000 넘어가더라도 정확도가 91% 이상을 넘기 어려움

In [ ]:
log=LogisticRegression(solver='lbfgs',max_iter=50,random_state=42)
log.fit(x_train,y_train)
pred_test=log.predict(x_test)
print('Accuracy in iterate',iter,'is',metrics.accuracy_score(y_test,pred_test))


* iter=50 일때 정확도 91.65 iter =100 보다 높음

>   ### 1.1.2 SVM
>   * Reference : https://liveyourit.tistory.com/62

In [3]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std= sc.transform(x_test)
svm=SVC(gamma='scale',random_state=42)
svm.fit(x_train_std, y_train)

SVC(random_state=42)

In [9]:
import pickle
import joblib
joblib.dump(svm,"svm_0830_MNIST.pkl")

['svm_0830_MNIST.pkl']

In [ ]:
SVM_pre=svm.predict(x_train_std)

In [52]:
print('accuracy_score is',metrics.accuracy_score(SVM_pre,y_train))

accuracy_score is 0.9863650793650793


> * Model regeneration with PCA

In [13]:
from os import times
from threading import Timer


Timer
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
x_train_PC=pca.fit_transform(x_train_std)
x_test_PC=pca.fit_transform(x_test_std)

svm_PC=SVC(gamma='scale',random_state=42)
svm_PC.fit(x_train_PC, y_train)


SVC(random_state=42)

In [14]:
joblib.dump(svm,"svm_0830_MNIST_PC.pkl")

['svm_0830_MNIST_PC.pkl']

>   >###  Standargation


>   ### 1.1.3 Decision tree


>   > ### Random forest
>   ## 1.2 Dimension Reduction
>   ## 1.3 Hyper parameter Tunning

## 2. Unsupervised learning


## 3. 기타 팁들

> ### __sklearn을 활용한 cuda 연동__
> * _Reference 1_ : __https://www.youtube.com/watch?v=GbC1BujV-J4__
> ### __딥러닝 환경 구축__
> * _Reference 2_ : _https://theorydb.github.io/dev/2020/02/14/dev-dl-setting-local-python/_